In [1]:
df = spark.read.format("csv").option("header","true").load("Files/Customer_Raw/customer_bank_a.csv")
# df now is a Spark DataFrame containing CSV data from "Files/Customer_Raw/customer_bank_a.csv".
display(df)

StatementMeta(, fc888c25-9919-4050-a026-3dc679d504df, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 889b46e5-ba3a-42f0-82ea-6710f49d0d9d)

In [11]:
from pyspark.sql.functions import sha2, col, concat_ws

# Tokenize by hashing the SSN (or Name+SSN for stronger uniqueness)
df_tokenized = df.withColumn("TokenizedID", sha2(concat_ws("SSN", "Name"), 256))

StatementMeta(, fc888c25-9919-4050-a026-3dc679d504df, 13, Finished, Available, Finished)

In [12]:
display(df_tokenized)

StatementMeta(, fc888c25-9919-4050-a026-3dc679d504df, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c6151b71-7581-43db-b96a-7c3f2a28e3d7)

In [13]:
df_tokenized.select("TokenizedID", "Region", "CustomerSince", "TotalLoans", "LatePayments") \
    .write.mode("overwrite").saveAsTable("customer")

StatementMeta(, fc888c25-9919-4050-a026-3dc679d504df, 15, Finished, Available, Finished)

In [14]:
df_cust = spark.sql("SELECT * FROM BankA_LH.customer LIMIT 1000")
display(df_cust)

StatementMeta(, fc888c25-9919-4050-a026-3dc679d504df, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0498093a-00a1-4a42-87b9-efd4eea18a44)

### Read data from Bureau using abfss path

In [16]:
creditscore_df = spark.read.load("abfss://ce45ca2c-6b31-4867-9e33-fc9ffa484d85@onelake.dfs.fabric.microsoft.com/873f13e1-2bf0-4797-b6bc-c51b787a3f98/Tables/cleanroom_dp_bureau_data")
display(creditscore_df)

StatementMeta(, fc888c25-9919-4050-a026-3dc679d504df, 18, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, df3968f0-bf14-41ba-b083-d0a90df1d5a6)

In [17]:
# Inner join on SSN
joined_df = bureau_df.join(df_tokenized, on="SSN", how="inner")

# Select the desired fields to build creditscore_df
creditscore_df = joined_df.select(
    df_tokenized.TokenizedID,
    bureau_df.DP_CreditScore,
    bureau_df.DP_NumDefaults,
    bureau_df.DP_InquiriesLast6Mo
)

# Show a few sample rows
creditscore_df.show()

StatementMeta(, fc888c25-9919-4050-a026-3dc679d504df, 19, Finished, Available, Finished)

+--------------------+-----------------+--------------------+--------------------+
|         TokenizedID|   DP_CreditScore|      DP_NumDefaults| DP_InquiriesLast6Mo|
+--------------------+-----------------+--------------------+--------------------+
|79b2e757d7c0df9fa...|625.8219649525404|   4.687609675993894|  0.3946523554639012|
|1561be5562449fd1f...|652.4229190473105|  1.0848322837012438|  3.8374448241460093|
|fbcd1ad1239c94c83...|625.7408042273983|  0.3729653081738717|  3.8722698379815768|
|8207b3040e516aea8...|828.5561928866085|   2.451400117487093|  -1.240864200137326|
|63cbeac1b8212f5ad...|626.7492861247579|  1.6571290451112353|   4.871589167619991|
|ca3d1c648666bea2b...|646.0059332969759|  1.7192620839296662|  3.5560420021129415|
|6ef7257c45a2b57f0...|642.7345215252027|   2.078774570220906|  -0.635634051415954|
|4e1de4ce054d8ca03...|764.3561587641764|   4.874622466431111|   5.273205895336675|
|42d11be07de98435f...|726.6116657044333|  0.7202530823590825|-0.19539151610651606|
|165